In [1]:
from auto_functions import *
import yaml
from py_sc_fermi.inputs import inputs_from_files
from pandas import DataFrame as df
import pandas as pd
from itertools import product
from tqdm import tqdm
import numpy as np
from scipy.constants import physical_constants

with open('automator_config.yaml', 'r') as stream:
    try:
        a = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

defects = import_calculations_from_file('defects.yaml')
elements = import_calculations_from_file('elements.yaml')
interest = import_calculations_from_file('interest.yaml')
        

def make_all_defects(config_dict,chem_pots):
    to_write = []
    defs = []
    for k,v in config_dict['defects'].items():
        print(k)
        label = [i for i in config_dict['defects'][k][1:]]
        to_calcs = [defects[i] for i in label]
        out = make_defect(to_calcs, elements, interest['LLZO'], delta_mu=chem_pots, corr=config_dict['iccs'], sites=1)
        out._nsites = config_dict['defects'][out.name][0]
        defs.append(out)
    return defs

ex_grid = (pd.read_csv('ex_grid.dat', skiprows=10, sep='\s+'))
grid = (pd.read_csv('grid.dat', skiprows=10, sep='\s+'))
all_defects = make_all_defects(a,ex_grid.iloc[5497].to_dict())

#print(all_defects)

a

v_O
v_Li
Li_i
O_i
v_La
v_Zr
Zr_i
Zr_Li_tet
Zr_Li
Zr_La
La_Zr
Li_La
Li_Zr
La_Li


{'defects': {'La_Li': [3, 'La_Li', 'La_Li_+', 'La_Li_++'],
  'La_Zr': [2, 'La_Zr', 'La_Zr_-'],
  'Li_La': [2, 'Li_La', 'Li_La_-', 'Li_La_--'],
  'Li_Zr': [1, 'Li_Zr', 'Li_Zr_-', 'Li_Zr_--', 'Li_Zr_---'],
  'Li_i': [1, 'Li_i', 'Li_i_+'],
  'O_i': [1, 'O_i', 'O_i_-'],
  'Zr_La': [2, 'Zr_La', 'Zr_La_+'],
  'Zr_Li': [2, 'Zr_Li', 'Zr_Li_+', 'Zr_Li_++', 'Zr_Li_+++'],
  'Zr_Li_tet': [1,
   'Zr_Li_tet',
   'Zr_Li_tet_+',
   'Zr_Li_tet_++',
   'Zr_Li_tet_+++'],
  'Zr_i': [1, 'Zr_i', 'Zr_i_+', 'Zr_i_++', 'Zr_i_+++', 'Zr_i_++++'],
  'v_La': [2, 'v_La_---'],
  'v_Li': [3, 'v_Li', 'v_Li_-'],
  'v_O': [3, 'v_O_++', 'v_O_+', 'v_O'],
  'v_Zr': [1, 'v_Zr_----']},
 'e_gap': 5.904,
 'e_vbm': 3.647,
 'iccs': {1: 0.054923318940534864,
  2: 0.21969327576213946,
  3: 0.4943098704648138,
  4: 0.8787731030485578,
  5: 1.3730829735133716,
  6: 1.9772394818592551,
  7: 2.6912426280862087},
 'nelect': 544,
 'potals': {'La_Li': 0,
  'La_Li_+': 0.4787106060606092,
  'La_Li_++': 0.480425000000003,
  'La_Zr': 0,
  'L

In [3]:
### The idea in this cell is to replicate the unrelaxed calculation, i.e. high T, nothing fixed

unitcell_filename = 'unitcell.dat'
totdos_filename = 'totdos.dat'
input_fermi_filename = 'input-fermi-frozen.dat'

inputs = inputs_from_files(unitcell_filename=unitcell_filename, 
                           input_fermi_filename=input_fermi_filename, 
                           totdos_filename=totdos_filename)


defect_system_unrelaxed = DefectSystem( defect_species=all_defects,
                                volume=inputs['volume'],
                                dos=inputs['dos'],
                                temperature=1500,
                                spin_pol = inputs['nspinpol'])

#defect_system.report()
constraint = (defect_system_unrelaxed.to_dict()['v_Li']) - (defect_system_unrelaxed.to_dict()['Li_i'])

defect_system_unrelaxed.report()

## These carrier concentrations are the same as Aron and Sunghyun's, so, I assume this is all still working 

Volume of cell: 1056.622886347127 A^3
Found spin polarised system...
Number of electrons in system: 544
Energy gap of system: 5.904 eV
Temperature: 1500.0 K
Number of defect species: 13
Reading spin-polarised DOS
SC Fermi level :      3.378588096089299  (eV)

Concentrations:
n (electrons)  : 4847943842054.927 cm^-3
p (holes)      : 44482918549.52475 cm^-3
v_O            : 86607201165009.05 cm^-3
v_Li           : 3.1684546577215955e+19 cm^-3
Li_i           : 3.168936670028114e+19 cm^-3
O_i            : 104811.65773031458 cm^-3
v_La           : 223517489101.706 cm^-3
v_Zr           : 157591.9823961873 cm^-3
Zr_i           : 1401347759.2466927 cm^-3
Zr_Li_tet      : 123190066116.46095 cm^-3
Zr_Li          : 2176065069891340.2 cm^-3
Zr_La          : 3062493908592986.5 cm^-3
La_Zr          : 900328239318469.5 cm^-3
Li_La          : 4344992355267241.0 cm^-3
Li_Zr          : 1652076119080942.5 cm^-3
La_Li          : 1545607990753.3008 cm^-3

Breakdown of concentrations for each defect charge 

/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/defect_charge_state.py:79: RuntimeWarning: overflow encountered in exp
  return self.degeneracy * np.exp(expfac)
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1762: RuntimeWarning: invalid value encountered in double_scalars
  p = (xf - fulc) * q - (xf - nfc) * r
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1763: RuntimeWarning: invalid value encountered in double_scalars
  q = 2.0 * (q - r)


In [4]:
# This cell is an excercise in what happens if we just let everything fall to 300 K with nothing fixed, essentially everything other then Li defects are totally wiped out 
defect_system_full_relax = DefectSystem( defect_species=all_defects,
                                volume=inputs['volume'],
                                dos=inputs['dos'],
                                temperature=300,
                                spin_pol = inputs['nspinpol'])
defect_system_full_relax.report()
defect_system_full_relax.write_inputs()

SC Fermi level :      3.435400610912699  (eV)

Concentrations:
n (electrons)  : 1.410777787112445e-21 cm^-3
p (holes)      : 2.8920215984735876e-37 cm^-3
v_O            : 1.0282294020879069e-19 cm^-3
v_Li           : 4424250209729.337 cm^-3
Li_i           : 4424258787811.561 cm^-3
O_i            : 1.5766234074935817e-59 cm^-3
v_La           : 3.1720433505619897e-26 cm^-3
v_Zr           : 7.961214104162664e-55 cm^-3
Zr_i           : 1.0168276604181081e-42 cm^-3
Zr_Li_tet      : 2.7266172353264954e-32 cm^-3
Zr_Li          : 5.2053723899453675e-12 cm^-3
Zr_La          : 2.330844971766325e-09 cm^-3
La_Zr          : 4.1492671933788866e-10 cm^-3
Li_La          : 9.779563656522404e-06 cm^-3
Li_Zr          : 1.1195706995312348e-05 cm^-3
La_Li          : 1.6744653532635308e-27 cm^-3

Breakdown of concentrations for each defect charge state:
---------------------------------------------------------
v_O        : Charge Concentration(cm^-3) Total
           :  2  9.969931e-20          96.96 
     

/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:86: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((e_fermi - edos)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:89: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((edos - e_fermi)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/defect_charge_state.py:79: RuntimeWarning: overflow encountered in exp
  return self.degeneracy * np.exp(expfac)
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1760: RuntimeWarning: invalid value encountered in double_scalars
  r = (xf - nfc) * (fx - ffulc)
/home/alex/anaconda3/lib/python3.6/site-packages/scipy

In [4]:
### This cell is where we start cheating the system by fixing things. This begins by fixing everything to the concentrations in the unrelaxed calculation

from py_sc_fermi.defect_charge_state import DefectChargeState, FrozenDefectChargeState
from py_sc_fermi.defect_system import DefectSystem
from py_sc_fermi.defect_species import DefectSpecies
from py_sc_fermi.inputs import inputs_from_files


for i in defect_system_unrelaxed.defect_species:
    for j in i.charge_states:
        c = i.charge_states[j].get_concentration(defect_system_unrelaxed.get_sc_fermi(verbose=False), defect_system_unrelaxed.temperature)
        i.charge_states[j]._fixed_concentration = c

print(defect_system_unrelaxed.to_dict()) ### This is just a sanity check comparison, to see if all the concentrations remain the same after fixing as they do before fixing
defect_system_unrelaxed.write_inputs()



/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/defect_charge_state.py:79: RuntimeWarning: overflow encountered in exp
  return self.degeneracy * np.exp(expfac)
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1762: RuntimeWarning: invalid value encountered in double_scalars
  p = (xf - fulc) * q - (xf - nfc) * r
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1763: RuntimeWarning: invalid value encountered in double_scalars
  q = 2.0 * (q - r)


{'Fermi Energy': 3.378588096089299, 'p0': 44482918549.52475, 'n0': 4847943842054.927, 'v_O': 86607201165009.05, 'v_Li': 3.1684546577215955e+19, 'Li_i': 3.168936670028114e+19, 'O_i': 104811.65773031458, 'v_La': 223517489101.706, 'v_Zr': 157591.9823961873, 'Zr_i': 1401347759.2466927, 'Zr_Li_tet': 123190066116.46095, 'Zr_Li': 2176065069891340.2, 'Zr_La': 3062493908592986.5, 'La_Zr': 900328239318469.5, 'Li_La': 4344992355267241.0, 'Li_Zr': 1652076119080942.5, 'La_Li': 1545607990753.3008}


In [6]:
# So now to try and replicate Sunghyun's analysis:

# first, reset the lithium defects
all_defects[1] = make_defect([defects['v_Li'],defects['v_Li_-']], elements, interest['LLZO'], delta_mu=ex_grid.iloc[5497].to_dict(), corr=a['iccs'], sites=3)
all_defects[2] = make_defect([defects['Li_i'],defects['Li_i_+']], elements, interest['LLZO'], delta_mu=ex_grid.iloc[5497].to_dict(), corr=a['iccs'], sites=1)

# then regenerate the defect system, 300K
defect_system_sunghyun = DefectSystem( defect_species=all_defects,
                                volume=inputs['volume'],
                                dos=inputs['dos'],
                                temperature=300,
                                spin_pol = inputs['nspinpol'])

# then fix the total defect concentrations of Li defects to be those of the high T calc
defect_system_sunghyun.defect_species_by_name('v_Li').fix_concentration( 0.3163805000637E+020)  # / 1e24 * defect_system_sunghyun.volume 
defect_system_sunghyun.defect_species_by_name('Li_i').fix_concentration( 0.3163815149663E+020) #  / 1e24 * defect_system_sunghyun.volume 
defect_system_sunghyun.report()

### This calculation seems to explode. 

defect_system_sunghyun.write_inputs()

SC Fermi level :      -27.577780507626215  (eV)

Concentrations:
n (electrons)  : 0.0 cm^-3
p (holes)      : 4.76915363262079e+23 cm^-3
v_O            : inf cm^-3
v_Li           : 2.994261284245561e+40 cm^-3
Li_i           : 2.9942708893999935e+40 cm^-3
O_i            : 1.5766234074935817e-59 cm^-3
v_La           : 0.0 cm^-3
v_Zr           : 0.0 cm^-3
Zr_i           : inf cm^-3
Zr_Li_tet      : inf cm^-3
Zr_Li          : inf cm^-3
Zr_La          : inf cm^-3
La_Zr          : 1.6243062956520694e-47 cm^-3
Li_La          : 1.1895618484225884e-102 cm^-3
Li_Zr          : 3.1458824244592735e-136 cm^-3
La_Li          : inf cm^-3

Breakdown of concentrations for each defect charge state:
---------------------------------------------------------
v_O        : Charge Concentration(cm^-3) Total
           :  2    inf          nan  [fixed]
           :  1    inf          nan  [fixed]
           :  0  6.418361e-38          0.00  [fixed]
---------------------------------------------------------
v_Li  

/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:86: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((e_fermi - edos)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:89: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((edos - e_fermi)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/defect_charge_state.py:79: RuntimeWarning: overflow encountered in exp
  return self.degeneracy * np.exp(expfac)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/defect_species.py:221: RuntimeWarning: invalid value encountered in double_scalars
  cs_concentrations[q] *= scaling
/mnt/c/Users/Alex/Materials_work/new_fermi/p

In [8]:
## reset v_li and li_i for ben-style analysis

all_defects[1] = make_defect([defects['v_Li'],defects['v_Li_-']], elements, interest['LLZO'], delta_mu=ex_grid.iloc[5497].to_dict(), corr=a['iccs'], sites=3)
all_defects[2] = make_defect([defects['Li_i'],defects['Li_i_+']], elements, interest['LLZO'], delta_mu=ex_grid.iloc[5497].to_dict(), corr=a['iccs'], sites=1)

defect_system_ben = DefectSystem( defect_species=all_defects,
                                volume=inputs['volume'],
                                dos=inputs['dos'],
                                temperature=300)

defect_system_ben.get_constrained_sc_fermi({'v_Li': +1, 'Li_i': -1}, (constraint / 1e24 * defect_system_ben.volume))

/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:86: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((e_fermi - edos)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:89: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((edos - e_fermi)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/defect_charge_state.py:79: RuntimeWarning: overflow encountered in exp
  return self.degeneracy * np.exp(expfac)
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1762: RuntimeWarning: invalid value encountered in double_scalars
  p = (xf - fulc) * q - (xf - nfc) * r
/home/alex/anaconda3/lib/python3.6/site-package

3.254589333978444

In [21]:
print((defect_system_ben.out))

544


In [11]:
a = defect_system_sunghyun.defect_species_by_name('v_Li')
a.charge_states[0].write_fixed_concentration(1200)

In [23]:
a.charge_states[0]._fixed_concentration = 1000000000

In [24]:
a.charge_states[0]._fixed_concentration

1000000000